# Notebook to extract LMC and SMC Cadence

In [312]:
# Check the version of MAF - the output should be version 2.4.2 or higher.
# 'unknown' means you're running from a git repo version,
# and have made changes to your local code

import lsst.sims.maf
lsst.sims.maf.__version__

'2.13.0.sims-59-g0a27faef'

In [313]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [314]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

from lsst.sims.maf.db import ResultsDb

In [315]:
import sys
print(sys.path)

['/Users/dagoret/MacOSX/GitHub/LSST/MyCadence/ExtractLMC', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/obs_lsst/19.0.0-80-g0bd2bb3+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_dateloc/master-g60a36c8d46+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_observatory_model/master-g3d307a5f45+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_almanac/master-g021b69e146+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_downtimeModel/master-g55f72efa65+17/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_cloudModel/master-ge3724df529+19/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/s

### Specify my other python path (my installation)

In [316]:
otherpythonpaths=['/Users/dagoret/anaconda3/lib/python37.zip', '/Users/dagoret/anaconda3/lib/python3.7', '/Users/dagoret/anaconda3/lib/python3.7/lib-dynload', '', '/Users/dagoret/.local/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/dagoret/.ipython']

In [317]:
for thepath in otherpythonpaths:
    sys.path.append(thepath) # go to parent dir

In [318]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Astronomy

In [319]:
gindex=1

In [320]:
Galaxy_Name=["lmc","smc"]

In [321]:
Galaxy_Delta_RA=[10.75,5.33]
Galaxy_Delta_DEC=[9.17,3.083]

In [322]:
GName=Galaxy_Name[gindex]
galaxy_delta_ra=Galaxy_Delta_RA[gindex]
galaxy_delta_dec=Galaxy_Delta_DEC[gindex]

## astroquery

In [323]:
galaxy_delta_ra=10.75
galaxy_delta_dec=9.17

In [324]:
import astroquery
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

In [325]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object(GName)
result_table.pprint(show_unit=True)

MAIN_ID      RA        DEC    ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"    "d:m:s"  ...                                            
-------- ---------- --------- ... -------- -------------- -------------------
NAME SMC 00 52 38.0 -72 48 01 ...        D              O 2003A&A...412...45P


In [326]:
Angle(result_table["RA"][0]+" hour")

<Angle 0.87722222 hourangle>

In [327]:
Angle(result_table["DEC"][0]+" degrees")

<Angle -72.80027778 deg>

In [328]:
galaxy_coord=SkyCoord(ra=result_table["RA"][0]+"hour",dec=result_table["DEC"][0]+ "degrees")
galaxy_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (13.15833333, -72.80027778)>

In [329]:
galaxy_ra=galaxy_coord.ra.deg
galaxy_dec=galaxy_coord.dec.deg

In [330]:
print(galaxy_ra,galaxy_dec)

13.158333333333331 -72.80027777777778


# Config

## Database selection

In [331]:
dbindex=2

In [332]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
colors = {'u': 'cyan', 'g': 'g', 'r': 'y','i': 'r', 'z': 'm', 'y': 'k'}
Nfilt=len(filters)

In [333]:
database=["baseline2018a.db","baseline_v1.5_10yrs.db","baseline_2snaps_v1.5_10yrs.db"]

In [334]:
dbfileName=database[dbindex]

In [335]:
dbtag=dbfileName.split(".")[0]

In [336]:
dbtag

'baseline_2snaps_v1'

# Prepare access to database

## SQL Selection

In [337]:
raCol = 'fieldRA'
decCol = 'fieldDec'

In [338]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [339]:
sqlWhere

'fieldRA < 18.533333 and fieldRA > 7.783333 and fieldDEC < -68.215278 and fieldDEC > -77.385278  '

## Metric

In [340]:
dbfields=['observationId','night','observationStartMJD','fiveSigmaDepth','filter','airmass','skyBrightness','seeingFwhmEff','fieldId','fieldRA','fieldDec']

In [341]:
metric=metrics.PassMetric(cols=dbfields)

##  Database to open

In [342]:
opsdb = db.OpsimDatabase('/Users/dagoret/DATA/Cadence/'+dbfileName)
outDir = 'output_galaxycadence'
resultsDb = db.ResultsDb(outDir=outDir)

## Slicer

In [343]:
slicer_uni=slicers.UniSlicer()
slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg='degrees')
nside = 512
slicer_heal = slicers.HealpixSlicer(nside=nside, lonCol=raCol, latCol=decCol)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


# Run

## Bundle

- Bundles dict for all filter

In [344]:
bundles = {}
bundles["LMC"] = metricBundles.MetricBundle(metric, slicer_uni, sqlWhere)
print(bundles["LMC"].dbCols)

{'fieldDec', 'skyBrightness', 'airmass', 'observationId', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'fieldId', 'filter', 'seeingFwhmEff'}


In [345]:
bgroup = metricBundles.MetricBundleGroup(bundles, opsdb, outDir=outDir,resultsDb=resultsDb)

In [346]:
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database SummaryAllProps with constraint fieldRA < 18.533333 and fieldRA > 7.783333 and fieldDEC < -68.215278 and fieldDEC > -77.385278   for columns ['airmass', 'skyBrightness', 'fieldDec', 'observationId', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'fieldId', 'filter', 'seeingFwhmEff']
Found 705 visits
Running:  ['LMC']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "fieldRA < 18.533333 and fieldRA > 7.783333 and fieldDEC < -68.215278 and fieldDEC > -77.385278  " constraint now.
Plotting complete.


/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_maf/2.13.0.sims-59-g0a27faef/python/lsst/sims/maf/db/resultsDb.py:279: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [347]:
print(bundles["LMC"].dbCols)

{'fieldDec', 'skyBrightness', 'airmass', 'observationId', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'fieldId', 'filter', 'seeingFwhmEff'}


In [348]:
print(len(bundles["LMC"].metricValues[0]))

705


# Output

In [349]:
pdfilename="cadence_"+GName+"_"+dbtag+".csv"

In [350]:
pdfilename

'cadence_smc_baseline_2snaps_v1.csv'

In [351]:
df  = pd.DataFrame()

In [352]:
N=len(dbfields)

In [353]:
for index in np.arange(N):
    fieldname=dbfields[index]
    data=bundles["LMC"].metricValues[0][fieldname]
    df[fieldname]=data

In [354]:
df

,observationId,night,observationStartMJD,fiveSigmaDepth,filter,airmass,skyBrightness,seeingFwhmEff,fieldId,fieldRA,fieldDec
0,2652,4,59857.287370,22.179584,y,1.373897,18.242457,0.779820,-1,15.413480,-69.750803
1,2654,4,59857.288266,22.104854,y,1.462966,18.203263,0.809771,-1,17.638771,-74.773566
2,2655,4,59857.288711,22.106954,y,1.448368,18.188760,0.804913,-1,11.309053,-72.525003
3,2685,4,59857.302138,21.820228,y,1.403919,18.239957,1.079491,-1,15.413480,-69.750803
4,2687,4,59857.303033,21.752114,y,1.487391,18.205737,1.117555,-1,17.638771,-74.773566
...,...,...,...,...,...,...,...,...,...,...,...
700,1858425,3374,63196.142067,22.198605,y,1.368825,18.164425,0.741057,-1,16.996810,-70.278866
701,1858452,3374,63196.154227,22.154884,y,1.460694,18.099848,0.739169,-1,14.015021,-73.216114
702,1858453,3374,63196.154676,22.075693,y,1.542032,18.085921,0.780227,-1,10.067607,-76.065195
703,1858457,3374,63196.156467,22.076568,y,1.450475,18.119416,0.802620,-1,8.723585,-70.510047


In [355]:
df.to_csv(pdfilename)

In [356]:
!ls

ExtractLMCCadence.ipynb            cadence_smc_baseline2018a.csv
cadence_lmc_baseline2018a.csv      cadence_smc_baseline_2snaps_v1.csv
cadence_lmc_baseline_2snaps_v1.csv cadence_smc_baseline_v1.csv
cadence_lmc_baseline_v1.csv        output_galaxycadence
